# Function Calling with the new Hermes

Copied and ajdusted from https://github.com/abacaj/openhermes-function-calling/tree/main
Model: teknium/OpenHermes-2.5-Mistral-7B


In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_id = "teknium/OpenHermes-2.5-Mistral-7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16,device_map="auto")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
import json
fn_call_syntax = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""

def generate_functions_prompt(query, functions=[]):
    func_string = "\n\n".join([json.dumps(fn) for fn in functions])
    prompt = f"""<|im_start|>system
You are a helpful assistant with access to the following functions:

{func_string}

To use these functions respond with:
<functioncall> {fn_call_syntax} </functioncall>

Edge cases you must handle:
- If there are no functions that match the user request, you will respond politely that you cannot help.<|im_end|>
<|im_start|>user
{query}<|im_end|>
<|im_start|>assistant"""
    return  prompt

def generate(prompt):
    return pipe(generate_functions_prompt(prompt),max_new_tokens=512,do_sample=False,return_full_text=False, pad_token_id=pipe.tokenizer.eos_token_id)[0]['generated_text']


In [ ]:
functions = [
    {
        "name": "call_uber",
        "description": "Find suitable ride for customers given the location, type of ride, and the amount of time the customer is willing to wait as parameters",
        "parameters":  [
            {"name": "loc", "type": "string", "description": "location of the starting place of the uber ride"},
            {"name":"type", "type": "string", "enum": ["plus", "comfort", "black"], "description": "types of uber ride user is ordering"},
            {"name": "time", "type": "number", "description": "the amount of time in minutes the customer is willing to wait"}
            ]
    },
    {
        "name": "get_current_weather",
        "description": "Gets the current weather for a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]
print(generate_functions_prompt("What is the weather in San Francisco?",functions))

In [ ]:
query = "What is the weather in New York?"
# get_current_weather last function in list
prompt = generate_functions_prompt(query,functions=functions)
print(generate(prompt))
# get_current_weather first function in list
prompt = generate_functions_prompt(query,functions=functions[::-1])
print(generate(prompt))
# Only get_current_weather in list
prompt = generate_functions_prompt(query,functions=[functions[::-1][0]])
print(generate(prompt))

In [ ]:
query = "Call me an plus Uber in Berkeley at zipcode 94704 in 10 minutes"
# call_uber first function in list
prompt = generate_functions_prompt(query,functions=functions)
print(generate(prompt))
# call_uber last function in list
prompt = generate_functions_prompt(query,functions=functions[::-1])
print(generate(prompt))
# Only call_uber in list
prompt = generate_functions_prompt(query,functions=[functions[0]])
print(generate(prompt))